In [2]:
import numpy as np
import pandas as pd
import os
import ast
import pdb
from openai._client import OpenAI

In [13]:
df = pd.read_csv('data/test_llama_formatted.csv')
df.head()

,data
0,"<s>[INST] Doctor: Hello, can you please tell m..."
1,"<s>[INST] Doctor: Hello, I understand that you..."
2,"<s>[INST] Doctor: Hello, we've received your r..."
3,"<s>[INST] Doctor: Hello, can you tell me what ..."
4,"<s>[INST] Doctor: Hello, I understand that you..."


In [9]:
from openai._client import OpenAI
client = OpenAI(api_key='sk-proj-XcfWDJqVthG3E1rbF5SRT3BlbkFJuDkqXMgC2HOOhIDhmhNi')

def embed(texts):
    embeddings = client.embeddings.create(
        input=texts,
        model='text-embedding-ada-002'
    )
    return [x.embedding for x in embeddings.data]

In [10]:
def get_embedding(text, model = 'text-embedding-ada-002'):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
get_embedding(df['data'].iloc[0])

In [15]:
df['data'].head(3).apply(get_embedding)

0    [-0.008033053949475288, 0.015747440978884697, ...
1    [-0.02702266164124012, 0.013304491527378559, 0...
2    [-0.018922461196780205, 0.010135144926607609, ...
Name: data, dtype: object

In [16]:
df['embeddings'] = df['data'].apply(get_embedding)
df.to_csv('data/train_llama_with_embeddings.csv', index=False)
df.to_pickle('data/train_llama_with_embeddings.pkl')

In [19]:
df = pd.read_pickle('data/train_llama_with_embeddings.pkl')
df.head()

,data,embeddings
0,"<s>[INST] Doctor: Hello, can you please tell m...","[-0.008033053949475288, 0.015747440978884697, ..."
1,"<s>[INST] Doctor: Hello, I understand that you...","[-0.02702266164124012, 0.013304491527378559, 0..."
2,"<s>[INST] Doctor: Hello, we've received your r...","[-0.018994081765413284, 0.010137662291526794, ..."
3,"<s>[INST] Doctor: Hello, can you tell me what ...","[-0.0067719705402851105, 0.004047193564474583,..."
4,"<s>[INST] Doctor: Hello, I understand that you...","[0.0007685507880523801, -0.010441986843943596,..."


In [20]:
question = 'Write SOAP based on the given data.'

In [21]:
question_embedding = get_embedding(question)
question, question_embedding[0:10], "..."

('Write SOAP based on the given data.',
 [-0.0021773893386125565,
  0.007757971063256264,
  -0.029985865578055382,
  -0.005458909552544355,
  0.006043662782758474,
  0.01994699239730835,
  -0.005778526421636343,
  -0.014317379333078861,
  0.010656313970685005,
  -0.01673629693686962],
 '...')

In [22]:
def fn(page_embedding):
  return np.dot(page_embedding, question_embedding)

df['distance'] = df['embeddings'].apply(fn)
df.head()

,data,embeddings,distance
0,"<s>[INST] Doctor: Hello, can you please tell m...","[-0.008033053949475288, 0.015747440978884697, ...",0.618067
1,"<s>[INST] Doctor: Hello, I understand that you...","[-0.02702266164124012, 0.013304491527378559, 0...",0.635855
2,"<s>[INST] Doctor: Hello, we've received your r...","[-0.018994081765413284, 0.010137662291526794, ...",0.615794
3,"<s>[INST] Doctor: Hello, can you tell me what ...","[-0.0067719705402851105, 0.004047193564474583,...",0.640064
4,"<s>[INST] Doctor: Hello, I understand that you...","[0.0007685507880523801, -0.010441986843943596,...",0.594527


In [24]:
df.sort_values('distance', ascending=False, inplace=True)
df.head()

,data,embeddings,distance
103,"<s>[INST] Doctor: Hi there, I'm Dr. Smith. I s...","[-0.007757121231406927, 0.01760244183242321, 0...",0.664621
146,"<s>[INST] Doctor: Hello, young lady. How can I...","[0.0011837020283564925, 0.024132462218403816, ...",0.663039
186,"<s>[INST] Doctor: Hello, I understand you have...","[-0.008206717669963837, -0.01169559732079506, ...",0.659467
153,"<s>[INST] Doctor: Hello, how can I help you to...","[0.0015383356949314475, 0.0010847680969163775,...",0.657644
127,"<s>[INST] Doctor: Hi, I understand you've been...","[-0.014724201522767544, 0.01606155000627041, 0...",0.657047


In [25]:
context = df['data'].iloc[0] + '\n' + df['data'].iloc[1] + '\n' + df['data'].iloc[2]
context

"<s>[INST] Doctor: Hi there, I'm Dr. Smith. I see you've been admitted to the Emergency Department due to agitation and an altered consciousness. Can you tell me when these symptoms started? Patient: Hey, doctor. They started just a few hours ago, and I've been feeling a progressive loss of verbal contact. Doctor: I understand. Your medical history shows that you have arterial hypertension, which is treated with atenolol. When you were admitted, your Glasgow Coma Score (GCS) was 9/15 with moderate neck stiffness. Your pupils were mid-size, reactive, and symmetric. Is that correct? Patient: Yes, that's correct. Doctor: Your vital signs showed a body temperature of 36.6°C, arterial blood pressure of 180/95 mm Hg, heart rate of 120/min, and respiratory rate of 60/min. We performed a brain computed tomography (CT) without iodinated contrast agent perfusion before the lumbar puncture, and it didn't reveal any significant abnormalities. Due to your progressive respiratory distress, intubatio

In [26]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are an assistant who is helping to write a SOAP using doctor-patient conversation"},
              {"role": "user", "content": question},
              {"role": "assistant", "content": f"Use this information from Doctor Patient Query to answer the user question: {context}. Please stick to the question while answering."}],
)

In [27]:
response.choices[0].message.content

"SOAP Note:\nSubjective:\nThe patient, admitted to the Emergency Department, reports symptoms of agitation and altered consciousness starting a few hours ago, with a progressive loss of verbal contact. The patient has a history of arterial hypertension treated with atenolol.\n\nObjective:\nUpon admission, the patient's Glasgow Coma Score (GCS) was 9/15, indicating moderate consciousness impairment, and has since worsened to 6/15. The patient exhibited moderate neck stiffness, mid-sized, reactive, symmetric pupils, elevated vital signs, and abnormal CSF findings consistent with bacterial meningitis caused by Streptococcus pneumoniae.\n\nAssessment:\nPrimary diagnosis is bacterial meningitis due to Streptococcus pneumoniae with complications of severe sepsis and respiratory distress. Differential diagnoses were considered but less likely based on CSF analysis and culture results.\n\nPlan:\nThe patient has been initiated on dexamethasone and ceftriaxone therapy for 14 days, with a follow-

In [28]:
def query(question):
  question_embedding = get_embedding(question)

  def fn(page_embedding):
    return np.dot(page_embedding, question_embedding)

  distance_series = df['embeddings'].apply(fn)
  top_four = distance_series.sort_values(ascending=False).index[0:4]

  text_series= df.loc[top_four]['data']
  context = '\n\n'.join(text_series)

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": "You are an assistant who is helping to write a SOAP using doctor-patient conversation"},
                {"role": "user", "content": question},
                {"role": "assistant", "content": f"Use this information from Doctor Patient Query to answer the user question: {context}. Please stick to the question while answering."}])
  return response.choices[0].message.content


In [29]:
query("Do you have any medical conditions?")

"The patient has a history of Parkinson's disease for about two years, surgical splenectomy, smoking, and hypertension. Additionally, the patient presents with a worsening cough, shortness of breath, significant weight loss, and difficulty swallowing. Upon examination, the patient was found to be hypoxemic and tachycardic, with imaging showing a right hilar mass involving the carina, subcarinal, right hilar, and paratracheal regions, along with other concerning findings. Biopsies revealed metastatic melanoma, along with B-cell lymphoma, indicating a complex oncological presentation. Treatment has been initiated with antibiotics for pneumonia and a multidisciplinary team consultation for comprehensive cancer management. Curated follow-up and support will be provided throughout the treatment process."